## Quiz #0801

### "Text Classification with Keras"

In [235]:
import numpy as np
import pandas as pd
import re
import nltk
import os
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM, Embedding
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import sequence
import string
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
nltk.download('stopwords')
nltk.download('punkt')
STOPWORDS = set(stopwords.words('english'))



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rania\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rania\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### Answer the following question by providing Python code:

1). Read in the movie review data from Cornell CS department. Carry out the EDA. <br>
- The data can be found [here](https://www.cs.cornell.edu/people/pabo/movie-review-data). <br>
- Download the “polarity dataset” and unzip. <br>
- Under the "txt_sentoken” folder, there are “pos” and “neg" subfolders. <br>

In [236]:
reviews = load_files('txt_sentoken/')
my_docs, y = reviews.data, reviews.target


2). Carry out the data preprocessing: <br>
- Cleaning.
- Stopword removal.

In [237]:
def clean_text(text):
    text = str(text)
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\n', '', text)
    text = text.replace('x', '')
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text


corpus= list(map(lambda x: clean_text(x), list(my_docs)))


3). Carry out label encoding by integers (required form by Keras):

In [238]:
# Make a dictionary with the top words.
n_words = 2000  
words = []
for i in range(len(corpus)):
    words += nltk.word_tokenize(corpus[i])
top_words = pd.Series(words).value_counts().index
top_words = top_words[0:n_words]                     # Apply a limitation.
my_dict = {}
my_dict_inv = {}
nbr = list(pd.Series(words).value_counts())
for i in range(len(top_words)):
    my_dict_inv[top_words[i]] = i
    my_dict[top_words[i]] = nbr[i]




In [239]:
# Convert the corpus into the label encoded form.
corpus_int =[]
for i in range(len(corpus)):
    words = nltk.word_tokenize(corpus[i])
    words2int = []
    for x in words:
        if x in my_dict:
            words2int += [my_dict_inv[x]]
    corpus_int.append(words2int)



4). Prepare the data for AI: <br>
- Apply the padding.
- Split the data into training and testing.

In [240]:
X = np.array(corpus_int)
y = np.array(y)

C:\Users\Rania\AppData\Local\Temp/ipykernel_13092/2831807499.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X = np.array(corpus_int)


In [241]:
# Padding: newswire lengths are uniformly matched to maxlen.
X = sequence.pad_sequences(X, maxlen = 100)
X.shape
# y is already binary. Thus, there is no need to covert to the one-hot-encoding scheme.

(2000, 100)

In [242]:
#split the data into training and testing
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)


5). Define the AI model (Embedding + LSTM):

In [243]:
x_train.shape

(1600, 100)

In [244]:
n_neurons = 100                    # Neurons within each memory cell.
n_input = 500                     # Dimension of the embeding space. 

In [245]:

my_model = Sequential()
my_model.add(Embedding(2000, n_input, input_length=100))
my_model.add(LSTM(n_neurons ))
my_model.add(Dense(1, activation='sigmoid'))


print(my_model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 500)          1000000   
_________________________________________________________________
lstm (LSTM)                  (None, 100)               240400    
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 1,240,501
Trainable params: 1,240,501
Non-trainable params: 0
_________________________________________________________________
None


6). Define the optimizer and compile the model:

In [246]:
n_epochs = 15                      # Number of epochs.
batch_size = 100                    # Size of each batch.
learn_rate = 0.0001 

7). Train the model and visualize the summary:

In [247]:
my_model.compile(loss='binary_crossentropy',
                 optimizer='RMSprop', metrics=['accuracy'])
history = my_model.fit(x_train, y_train, epochs=n_epochs, batch_size=batch_size,validation_split=0.1)


Epoch 1/15
15/15 [==============================] - 10s 543ms/step - loss: 0.6948 - accuracy: 0.5653 - val_loss: 0.6319 - val_accuracy: 0.7000
Epoch 2/15
15/15 [==============================] - 8s 530ms/step - loss: 0.4739 - accuracy: 0.8292 - val_loss: 0.4107 - val_accuracy: 0.8125
Epoch 3/15
15/15 [==============================] - 8s 531ms/step - loss: 0.2856 - accuracy: 0.8840 - val_loss: 0.5414 - val_accuracy: 0.7625
Epoch 4/15
15/15 [==============================] - 8s 549ms/step - loss: 0.1808 - accuracy: 0.9312 - val_loss: 0.5365 - val_accuracy: 0.8062
Epoch 5/15
15/15 [==============================] - 8s 527ms/step - loss: 0.1090 - accuracy: 0.9674 - val_loss: 0.5407 - val_accuracy: 0.7812
Epoch 6/15
15/15 [==============================] - 8s 526ms/step - loss: 0.0506 - accuracy: 0.9889 - val_loss: 0.5959 - val_accuracy: 0.7875
Epoch 7/15
15/15 [==============================] - 8s 528ms/step - loss: 0.0666 - accuracy: 0.9792 - val_loss: 0.6330 - val_accuracy: 0.7812
Epoch

8). Display the test result (accuracy):

In [249]:
score = my_model.evaluate(x_test, y_test, verbose=0)
print('Accuracy: {:0.2f}'.format(score[1]))


Accuracy: 0.77
